# WEB SCRAPING AND TEXT ANALYSIS 
* POSITIVE SCORE
* NEGATIVE SCORE
* POLARITY SCORE
* SUBJECTIVITY SCORE
* AVG SENTENCE LENGTH
* PERCENTAGE OF COMPLEX WORDS
* FOG INDEX
* AVG NUMBER OF WORDS PER SENTENCE
* COMPLEX WORD COUNT
* WORD COUNT
* SYLLABLE PER WORD
* PERSONAL PRONOUNS
* AVG WORD LENGTH

In [230]:
import numpy as np
import pandas as pd
import re
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [231]:
data = pd.read_excel("D:/DOWNLOAD/Output Data Structure.xlsx")
data

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40.0,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,147.0,https://insights.blackcoffer.com/the-future-of...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
111,148.0,https://insights.blackcoffer.com/big-data-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112,149.0,https://insights.blackcoffer.com/business-anal...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## PREPARING STOP WORDS

In [232]:
stop_words = []
addresses = ["D:/DOWNLOAD/StopWords_Auditor.txt", "D:/DOWNLOAD/StopWords_Currencies.txt", "D:/DOWNLOAD/StopWords_DatesandNumbers.txt",
          "D:/DOWNLOAD/StopWords_Generic.txt", "D:/DOWNLOAD/StopWords_GenericLong.txt", "D:/DOWNLOAD/StopWords_Geographic.txt", 
          "D:/DOWNLOAD/StopWords_Names.txt"]
for address in tqdm(addresses):
    df = pd.read_csv(address, header = None, encoding='latin1', on_bad_lines="skip", sep = "|")[0]
    for i in range(len(df)):
        word = df[i]
        if type(word) == str:
            stop_words.append(word.lower())
        else:
            stop_words.append(word)

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 50.74it/s]


## PREPARING POSITIVE AND NEGATIVE KEYWORDS

In [233]:
positive_keywords = pd.read_csv("D:/DOWNLOAD/positive-words.txt", header=None)
positive_keywords = np.squeeze(np.array(positive_keywords))

negative_keywords = pd.read_csv("D:/DOWNLOAD/negative-words.txt", header = None, encoding='latin1', on_bad_lines="skip", sep = "|")
negative_keywords = np.squeeze(np.array(negative_keywords))

In [234]:
# A FUNCTION THAT RETURNS THE SYLLABLE COUNT IN A WORD
def syllable_count(word):
    if len(word)>1:
        word = word.lower()
        count = 0
        vowels = "aeiouy"
        if word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith("e"):
            count -= 1
        if count == 0:
            count += 1
    else:
        count = 0
    return count

## OUR MAIN FUNCTION FOR WEB SCRAPING AND PROCESSING

In [235]:
def main_function(url):
    '''
    A FUNCTION THAT TAKES THE URL OF A WEBSITE AS AN INPUT AND SCRAPES IN TO EXTRACT CONTENT AND DO
    ALL THE NECESSARY MANIPULATIONS AND PRE PROCESSINGS TO RETURN A DICTIONARY OF REQUIRED OUTPUTS.
    '''
    r = requests.get(url)
    
    # PREPARING PARAGRAPHS
    paragraphs = []
    htmlParse = BeautifulSoup(r.content, 'html5lib')   
    for para in htmlParse.find_all("p"):
        paragraph = para.get_text().replace("\n", " ")
        paragraphs = paragraphs + [paragraph]
    
    # PREPARING WORDS AND SENTENCES
    words = []
    sentences = []
    
    for para in (paragraphs):
        for word in para.split(sep = " "): # SPACE TO SEGREGATE WORDS
            words.append(word)
        for sentance in para.split(sep = "."): # FULL STOP TO SEGREGATE SENTENCES
            sentences.append(sentance)
            
    filtered_words = [w for w in words if not w.lower() in stop_words]
    word_count = len(filtered_words)
#  ------------------------------------------------------------------------------------------------------------------------------
    positive_score, negative_score, complex_word_count, average_word_len, syll_count = 0, 0, 0, 0, 0
    
    for i in (range(0, len(words))):
        word = words[i] # CHECKING EACH INDIVIDUAL WORD FOR EACH SPECIFIED CONDITION.
        # POSITIVE KEYWORDS
        if word in positive_keywords:
            positive_score += 1
        # NEGATIVE KEYWORDS
        if word in negative_keywords:
            negative_score += 1
        # SYLLABLE COUNT
        s_count = syllable_count(word)
        syll_count += s_count
        if s_count > 2:
            complex_word_count += 1
        # AVERAGE WORD LENGTH
        average_word_len += len(word)
    
    # Sentimental Analysis
    polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score)/ ((words_after_cleaning) + 0.000001)
    
    # Analysis of Readability
    avg_sentence_len = len(words)/len(sentences)
    complex_words_percent = complex_word_count/ len(words)
    fog_index = 0.4 * (avg_sentence_len + complex_words_percent)
    
    # Average Number of Words Per Sentence
    avg_words_per_sentence = len(words)/len(sentences)
    
    # Syllable Count Per Word
    syll_count /= len(words)
    
    # PRNOUNS 
    pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
    pronoun_count = 0
    for sentence in sentences:
        pronouns = pronounRegex.findall(sentence)
        pronoun_count += len(pronouns)
    
    # Average Word Length
    average_word_len /= len(words) 

    
    polarity_score = (positive_score - negative_score)/ ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score)/ ((words_after_cleaning) + 0.000001)
    
    outputs = {
        "POSITIVE SCORE":positive_score, "NEGATIVE SCORE": negative_score,"POLARITY SCORE": polarity_score,
        "SUBJECTIVITY SCORE": subjectivity_score, "AVG SENTENCE LENGTH": avg_sentence_len,
        "PERCENTAGE OF COMPLEX WORDS": complex_words_percent, "FOG INDEX":fog_index, "AVG NUMBER OF WORDS PER SENTENCE": avg_words_per_sentence, 
        "COMPLEX WORD COUNT": complex_word_count, "WORD COUNT":  word_count, "SYLLABLE PER WORD": syll_count ,
        "PERSONAL PRONOUNS": pronoun_count, "AVG WORD LENGTH": average_word_len            
                }
    return outputs

## ITERATING OVER EACH WEBSITE AND PERFORMING ALL STEPS

In [237]:
for i in tqdm(range(0, len(data))):
    outputs = main_function(data["URL"][i])
    output_df = pd.DataFrame(outputs, index = [0])
    for j in output_df.columns:
        data[j][i] = output_df[j][0]

100%|████████████████████████████████████████████████████████████████████████████████| 114/114 [08:25<00:00,  4.43s/it]


## FINAL OUTPUT

In [238]:
data

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,71.0,26.0,0.463918,0.1552,14.923664,0.259847,6.073404,14.923664,508.0,1102.0,1.896164,3.0,5.807161
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,57.0,28.0,0.341176,0.1360,14.675926,0.184227,5.944061,14.675926,292.0,735.0,1.636593,9.0,5.092114
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,67.0,29.0,0.395833,0.1536,14.030075,0.247053,5.710851,14.030075,461.0,968.0,1.851018,5.0,5.563237
3,40.0,https://insights.blackcoffer.com/will-machine-...,56.0,14.0,0.600000,0.1120,13.216418,0.172784,5.355681,13.216418,306.0,779.0,1.638622,19.0,5.009599
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,52.0,19.0,0.464789,0.1136,14.181818,0.201389,5.753283,14.181818,377.0,928.0,1.696047,18.0,5.292201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,146.0,https://insights.blackcoffer.com/blockchain-fo...,27.0,23.0,0.080000,0.0800,11.703297,0.222535,4.770333,11.703297,237.0,559.0,1.776526,11.0,5.582160
110,147.0,https://insights.blackcoffer.com/the-future-of...,36.0,8.0,0.636364,0.0704,14.500000,0.208464,5.883386,14.500000,266.0,663.0,1.724138,4.0,5.378527
111,148.0,https://insights.blackcoffer.com/big-data-anal...,32.0,35.0,-0.044776,0.1072,11.837838,0.219178,4.822806,11.837838,288.0,716.0,1.732877,4.0,5.243531
112,149.0,https://insights.blackcoffer.com/business-anal...,29.0,2.0,0.870968,0.0496,12.457627,0.262585,5.088085,12.457627,193.0,413.0,1.891156,2.0,5.783673


In [240]:
data.to_csv("D:/INTERN/Output.csv", index = False)

# THANKYOU